In [63]:
# -*- coding: utf-8 -*-

#!pip3 install rdflib
#!pip3 install treelib

# https://github.com/openeduhub/oeh-metadata-vocabs/blob/master/oehTopics.ttl

import re, rdflib,json
from treelib import Node, Tree
from rdflib.namespace import RDF, SKOS
from rdflib import URIRef
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('german')).union(set(stopwords.words('english')))


import time


class TopicAssistent:

    def normalize(self, s):
        s = re.sub('[^A-Za-z0-9öüäÖÄÜß]+', ' ', s)
        return s.lower()


    def __init__(self):

        # create a RDF graph
        g = rdflib.Graph()

        # parse in an RDF file hosted on the Internet
        #result = g.parse("https://raw.githubusercontent.com/openeduhub/oeh-metadata-vocabs/master/oehTopics.ttl", format="ttl")
        result = g.parse("oehTopics.ttl", format="ttl")

        tree = Tree()
        #find top level node
        for s, p, o in g.triples((None, RDF.type, SKOS.ConceptScheme)):
            #print (s, p, o)
            tree.create_node("WLO", s, data={'w':0, 'uri': s})
            for s2, p2, o2 in g.triples((s, SKOS.hasTopConcept, None)):
                #print (s2, p2, o2)
                tree.create_node(o2, o2, parent=s, data={'w':0, 'uri': o2})
                #break
            
        foundSth = True
        #für jeden Knoten finde Kindknoten
        while foundSth:
            foundSth = False
            #print (len(tree))
            for node in tree.all_nodes():
                #print (node.tag)
                n = URIRef(node.tag)
                for s, p, o in g.triples((None, SKOS.broader, n)):
                    #print (s, tree.contains(s))
                    if not tree.contains(s):
                        tree.create_node(s, s, parent=node, data={'w':0})
                        foundSth = True


        for node in tree.all_nodes():
            for s, p, o in g.triples(( URIRef(node.identifier) , SKOS.prefLabel, None)):
                node.tag=o
                node.data['label']=o
                #print (o)


        ## create list of keywords
        keywords={}
        for s, p, o in g.triples((None, URIRef("https://schema.org/keywords"), None)):
            #print (s, o)
            n = self.normalize(o)
            if len(n)>2:
                try:
                    keywords[s].append(n)
                except:
                    keywords[s]=[]
                    keywords[s].append(n)

        # prefLabel
        for s, p, o in g.triples(( None , SKOS.prefLabel, None)):
            n = self.normalize(o)
            #print (n)
            if len(n)>2:
                try:
                    if not n in keywords[s]:
                        keywords[s].append(n)
                except:
                    keywords[s]=[]
                    keywords[s].append(n)
            
        self.keywords = keywords
        self.tree = tree


    def go(self, exampleText):
        start = time.process_time()
        newTree = Tree(self.tree, deep=True)
        print("init: ", time.process_time() - start)
            
        start = time.process_time()
        t=[]
        for tok in self.normalize(exampleText).split(' '):
            if not tok in STOPWORDS:
                t.append(tok)
        ntext = " " + " ".join(t) + " "
        #print (ntext)
        print("stopwords: ", time.process_time() - start)
        
        start = time.process_time()
        for c in self.keywords.keys():
            for k in self.keywords[c]:
                if ntext.find(" " + k + " ")>-1 :                        
                    newTree.get_node(c).data['w']=newTree.get_node(c).data['w']+1
                    newTree.get_node(c).data['match']=k 
        print("keywords: ", time.process_time() - start)
        
        start = time.process_time()
        # propagate data to the root
        for d in range (newTree.depth(), -1, -1):    
            #print ("L", d)
            for node in newTree.all_nodes():
                if d == newTree.depth(node):
                    if node.data!=None and node.data['w'] > 0:  
                        p = newTree.parent(node.identifier)
                        if p:
                            p.data['w'] = p.data['w'] + node.data['w']
        print("propergate: ", time.process_time() - start)

        #calcTree = Tree(newTree, deep=True)
        #print (len(calcTree))
        start = time.process_time()
        for node in newTree.all_nodes(): 
            #print (node, node.is_root())
            if node and not node.is_root() and node.data!=None and node.data['w'] == 0:
                #print (node.data, newTree.parent(node.identifier).data)
                if (newTree.contains(node.identifier)):
                    newTree.remove_node(node.identifier)
            else:
                if node and not node.is_root() and node.data!=None:
                    node.tag = node.tag + " (" + str(node.data['w']) + ")"
                    if 'match' in node.data.keys():
                        node.tag = node.tag + " [" + node.data['match'] + "]"
        print("clear: ", time.process_time() - start)
        start = time.process_time()

        newTree.show(key=lambda node: node.data["w"], reverse=True, idhidden=True) 
        
        print("show: ", time.process_time() - start)
        #return json.dumps(calcTree.to_dict(with_data=True, key=lambda node: node.data["w"], sort=True, reverse=True))

T=TopicAssistent()
        

In [64]:
start = time.process_time()
T.go("Im Englisch Unterricht behandeln wir heute Verben, Past Perfect und False Friends")
print("total: ", time.process_time() - start)

init:  0.3646789999999953
stopwords:  3.300000000194814e-05
keywords:  0.007322000000002049
propergate:  0.25916499999999587
clear:  0.08443800000000579
WLO
├── Englisch (5) [englisch]
│   ├── Grammatik (3) [grammatik]
│   │   └── Verben (2) [verben]
│   │       └── Past (1) [past]
│   └── Sprache und Aussprache (1)
│       └── False friends (1) [false friends]
├── Deutsch als Zweitsprache (4)
│   ├── Grammatik (3) [grammatik]
│   │   ├── Adverbien (1)
│   │   │   └── Temporaladverbien (1) [heute]
│   │   └── Verben (1) [verben]
│   └── Wortschatz (1)
│       └── Schule und Studium (1) [englisch]
├── Türkisch (2)
│   └── Grammatik (2) [grammatik]
│       └── Verben (1) [verben]
├── Spanisch (2)
│   └── Grammatik (2) [grammatik]
│       └── Verben (1) [verben]
└── Deutsch (1)
    └── Grammatik und Sprache untersuchen (1)
        └── Wortarten (1)
            └── Verben (1) [verben]

show:  0.0004189999999795191
total:  0.7168759999999992


In [65]:
T.go("Lineare Abhängigkeit / Unabhängigkeit Lineare Abhängigkeit _ Unabhängigkeit  Linearkombination linear abhängig Nullvektor Gauß Algorithmus Determinante")

init:  0.3466449999999952
stopwords:  5.099999999913507e-05
keywords:  0.009220999999982382
propergate:  0.27414699999999925
clear:  0.10268999999999551
WLO
├── Mathematik (3)
│   └── Analytische Geometrie (3)
│       ├── Lagebeziehungen (1)
│       │   └── Lineares Gleichungssystem (1) [gauß]
│       ├── Lineares Gleichungssystem (1) [gauß]
│       └── Vektorrechnung (1)
│           └── Lineare Abhängigkeit (1) [lineare abhängigkeit]
├── Englisch (2)
│   └── Themen und Wortschatz (2)
│       └── Geschichte (2)
│           ├── Geschichte der USA (1) [unabhängigkeit]
│           └── wichtige Ereignisse (1) [unabhängigkeit]
└── Medienbildung (1)
    └── Problemlösen, Handeln und Modellieren (1)
        └── Modellieren und Programmieren (1) [algorithmus]

show:  0.0004409999999950287


In [66]:
T.go("Dieselmotoren und Stickoxide Dieselmotoren und Stickoxide   AdBlue Urethan Stickoxid NO NOx NO2 N2O N2O4 Fahrverbot VW Diesel-Skandal Feinstaub")

init:  0.3300780000000003
stopwords:  5.000000001587068e-05
keywords:  0.009317999999993276
propergate:  0.2837999999999994
clear:  0.17435100000000148
WLO
└── Chemie (15)
    ├── Chemie und Umwelt (14)
    │   ├── Dieselmotoren und Stickoxide (10) [nox]
    │   ├── Luft und Luftverschmutzung (3) [feinstaub]
    │   │   └── Die Ozon-Problematik (1) [stickoxide]
    │   └── Chemie in der Landwirtschaft (1) [skandal]
    └── Organische Chemie (1)
        └── Die Kohlenwasserstoffe als Grundstoffe in der Organischen Chemie (1)
            └── Eigenschaften der  Kohlenwasserstoffe (1) [diesel]

show:  0.0002950000000083719


In [67]:
T.go("Elli Online — Voll daneben! - Cybermobbing_1595499758626 Elli passiert im Sportunterricht ein harmloses Missgeschick. Dieser Vorfall ist Auslöser für eine erschreckende Entwicklung. Elli wird gemobbt. Es beginnt mit anonymen Beleidigungen im Netz. Daraus entwickelt sich eine Welle von Hohn und Spott, die Elli überrollt. Nirgends kann sie den Beschimpfungen entfliehen, nicht im Internet und nicht in der Schule. Aber Elli ist nicht alleine. Sie hat Cosmo, ihre Freunde, ihre Familie und eine engagierte Lehrerin. Gemeinsam beratschlagen sie, was sie tun können, um das Mobbing zu stoppen. In der Realwelt und im Netz. (Online-Signatur Medienzentren: 4985739) Schüler-Schüler-Beziehung Internet")

init:  0.33034800000000075
stopwords:  0.0001160000000055561
keywords:  0.011819999999985953
propergate:  0.29611900000000446
clear:  0.08611299999999744
WLO
├── Religion (5)
│   ├── Ethik (3)
│   │   └── Verantwortung im Konflikt (3)
│   │       └── Beispiele für Gerechtigkeit und Ungerechtigkeit (3) [familie]
│   └── Menschsein und die Suche nach Gott (2)
│       └── Identität und Beziehung (2)
│           └── Beziehung (2) [beziehung]
├── Sport (4)
│   └── Ballsport (4)
│       ├── Indiaca (1) [netz]
│       ├── Badminton (1) [netz]
│       ├── Tennis (1) [netz]
│       └── Volleyball (1) [netz]
├── Deutsch als Zweitsprache (3)
│   └── Wortschatz (3)
│       ├── Miteinander, Freunde und Familie (2) [familie]
│       └── Schule und Studium (1) [schule]
├── Englisch (3)
│   └── Themen und Wortschatz (3)
│       ├── soziale Beziehungen (2)
│       │   ├── soziale Medien (1) [mobbing]
│       │   └── Familie (1) [familie]
│       └── Medien (1)
│           └── Internet (1) [internet]
├─